In [53]:
import warnings
from collections.abc import Iterable

import numpy as np

# np.seterr(divide='ignore')
# warnings.filterwarnings('ignore')
warnings.simplefilter('error')

class IGN:
    def __init__(self,z,y,dec_places=2):
        # Obs probs 
        self.z = z
        # Forecast probs 
        self.y = y
        # Number of times 
        self.T = len(self.z)
        # Base-rate 
        self.z_bar = np.mean(self.z)
        # Unique probabilities 
        self.I = sorted(set(np.round(self.y, dec_places)))
        
        assert np.max(self.y) < 1.0
        assert np.min(self.y) > 0.0
        self.eps = 1E-8
        
    @staticmethod
    def __compute_dkl(p,q,eps):
        dkl_score = (p * np.log2(p+eps/q+eps)) + ((1-p) * np.log2((1-p+eps)/(1-q+eps)))
        return np.mean(dkl_score)
    
    def compute_ign(self,from_components=False):
        if from_components:
            UNC = self.compute_unc() 
            REL = self.compute_rel()
            RES = self.compute_res()
            return (UNC - RES + REL)
        # ign = -np.sum(np.log2(np.abs(self.y - (1-self.z))))
        # return ign/self.T
        ign = self.__compute_dkl(self.z, self.y, self.eps)
        return ign 
    
    def compute_rel(self):
        rel_score = 0
        for yi in self.I:
            y_subset = self.y[self.y == yi]
            num_fcsts = len(y_subset)
            pyi = num_fcsts/self.T
            z_subset = self.z[self.y == yi]
            if len(z_subset) == 0:
                continue
            zi_bar = np.mean(z_subset)
            rel_score += (pyi * ( (zi_bar * np.log2( zi_bar+self.eps  /   yi+self.eps  )) + ((1-zi_bar)*np.log2(   (1-zi_bar+self.eps)/   (1-yi+self.eps   )) ) ) )
        return rel_score 
        
    def compute_res(self):
        res_score = 0
        for yi in self.I:
            y_subset = self.y[self.y == yi]
            num_fcsts = len(y_subset)
            pyi = num_fcsts/self.T
            z_subset = self.z[self.y == yi]
            if len(z_subset) == 0:
                continue
            zi_bar = np.mean(z_subset)
            res_score += (pyi * ( (zi_bar * np.log2( zi_bar+self.eps  /   self.z_bar+self.eps  )) + ((1-zi_bar)*np.log2(   (1-zi_bar+self.eps)/   (1-self.z_bar+self.eps   )) ) ) )
        return res_score        
    
    def compute_unc(self):
        return (-self.z_bar * np.log2(self.z_bar + self.eps)) - ((1-self.z_bar)*np.log2(1-self.z_bar+self.eps))
    
    @classmethod
    def compute_info_gain(cls, o, f1, f2, eps, from_components=False):
        DKL1 = cls.__compute_dkl(o, f1, eps)[1]
        DKL2 = cls.__compute_dkl(o, f2, eps)[1]
        return DKL1 - DKL2

    def compute_bs(self):
        return np.mean((self.z-self.y)**2)

    def compute_bss(self):
        bs = self.compute_bs()
        bs_unc = self.z_bar * (1-self.z_bar)
        return 1 - (bs/bs_unc)

    def compute_skill_score(self, return_components=True):
        U = self.compute_unc()
        R = self.compute_rel()
        D = self.compute_res()
        D_ss = D/U
        R_ss = R/U
        SS = D_ss - R_ss
        if return_components:
            return SS, D_ss, R_ss
        return SS

    def compute_info_gain_over_climo(self):
        R = self.compute_rel()
        D = self.compute_res()
        # Info gain over using the base-rate for forecasts
        return (D - R)

In [54]:
def generate_time_series(seed=42,num_samples=10000):
    rng = np.random.default_rng(seed=seed)

    o = rng.choice([0, 1], size=num_samples, p=[0.8, 0.2])
    
    f = np.zeros(num_samples)
    
    for i in range(num_samples):
        if o[i] == 1:
            f[i] = rng.choice(np.arange(0.40, 0.995, 0.01))
        else:
            f[i] = rng.choice(np.arange(0.01, 0.60, 0.01))
    return o, f

o, f = generate_time_series()
print(o)
print(f)

[0 0 1 ... 0 0 0]
[0.03 0.43 0.52 ... 0.18 0.44 0.51]


In [55]:
ign = IGN(o,f)

print(ign.compute_unc())
print(ign.compute_res())
print(ign.compute_rel())
print(ign.compute_ign())
print(ign.compute_ign(from_components=True))


0.7126321123563446
0.22283304791008754
0.39638657255553217
0.4553202605521375
0.8861856370017893


In [56]:
bs = ign.compute_bs()
bss = ign.compute_bss()
print(f"{bs=}, {bss=}")

SS, D_ss, R_ss = ign.compute_skill_score(return_components=True)
print(f"{SS=}, {D_ss=}, {R_ss=}")


bs=0.12144117999999998, bss=0.2275659838222952
SS=-0.24353873707933738, D_ss=0.3126901581424415, R_ss=0.5562288952217789
